In [1]:
import os
import pandas as pd

# Dataset path
dataset_path = "/kaggle/input/fruit-quality-classification"

# Empty list to store data
data = []

# Iterate through the dataset directory
for quality in os.listdir(dataset_path):  # Bad Quality_Fruits, Good Quality_Fruits, Mixed Qualit_Fruits
    quality_path = os.path.join(dataset_path, quality)
    if os.path.isdir(quality_path):  # Ensure it's a directory
        for fruit_folder in os.listdir(quality_path):  # Apple_Bad, Banana_Good, etc.
            fruit_path = os.path.join(quality_path, fruit_folder)
            if os.path.isdir(fruit_path):
                # Extract fruit name and quality from the folder name
                fruit = fruit_folder.rsplit("_", 1)[0]  # Extracts "Apple", "Banana", etc.
                for img in os.listdir(fruit_path):  # Iterate through images
                    img_path = os.path.join(fruit_path, img)
                    if img.lower().endswith(('.png', '.jpg', '.jpeg')):  # Ensure it's an image
                        data.append([img_path, fruit, quality])

# DataFrame
df = pd.DataFrame(data, columns=["image_path", "fruit", "quality"])
df

,image_path,fruit,quality
0,/kaggle/input/fruit-quality-classification/Goo...,Apple,Good Quality_Fruits
1,/kaggle/input/fruit-quality-classification/Goo...,Apple,Good Quality_Fruits
2,/kaggle/input/fruit-quality-classification/Goo...,Apple,Good Quality_Fruits
3,/kaggle/input/fruit-quality-classification/Goo...,Apple,Good Quality_Fruits
4,/kaggle/input/fruit-quality-classification/Goo...,Apple,Good Quality_Fruits
...,...,...,...
19521,/kaggle/input/fruit-quality-classification/Bad...,Apple,Bad Quality_Fruits
19522,/kaggle/input/fruit-quality-classification/Bad...,Apple,Bad Quality_Fruits
19523,/kaggle/input/fruit-quality-classification/Bad...,Apple,Bad Quality_Fruits
19524,/kaggle/input/fruit-quality-classification/Bad...,Apple,Bad Quality_Fruits
